In [2]:
import pandas as pd
url = 'https://github.com/Alf-caput/LAB02_ReglasYPatronesSecuenciales/raw/main/Online%20Retail.xlsx'
df = pd.read_excel(url)
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   541909 non-null  int64         
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 33.1+ MB


In [4]:
print(f'Número total de NA en el dataframe: {(col_na:=df.isna().sum()).sum()}')
pd.DataFrame({'Valores NA': col_na})

Número total de NA en el dataframe: 1454


,Valores NA
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,0
Country,0


In [5]:
(
    df
    .describe()
    .drop('count')
    .T
)

,mean,min,25%,50%,75%,max,std
Quantity,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,15287.518434,12346.0,14367.0,15287.0,16255.0,18287.0,1484.746041


Nos fijamos que existen valores negativos tanto en cantidad (Quantity) como en precio de la unidad (UnitPrice).  
Veamos cuantas entradas tienen este problema para estudiar que hacer.

In [23]:
df_negs = df.loc[(df.loc[:, 'Quantity'] <= 0) | (df.loc[:, 'UnitPrice'] <= 0), :]
df_negs.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548,United Kingdom


In [24]:
len(df_negs)

11805

Vemos que existen más de 11 mil entradas con valores negativos en alguna de las columnas "Quantity" y "UnitPrice".  

In [31]:
negs_ratio = round(len(df_negs)/len(df), 2)
print(negs_ratio)

0.02


Vemos que solo el 2% de los datos contienen valores negativos, consideramos que es una parte pequeña de los datos y por tanto los eliminamos del dataset.

In [32]:
df_std = (
    pd.concat([df, df_negs])
    .drop_duplicates(keep=False)
)
df_std.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


Comprobamos que hemos realizado correctamente la transformación.

In [37]:
df_negs = df_std.loc[(df_std.loc[:, 'Quantity'] <= 0) | (df_std.loc[:, 'UnitPrice'] <= 0), :]
len(df_negs)

0

In [7]:
df.loc[df.loc[:, 'Description'].isna(), :].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,2010-12-01 11:52:00,0.0,15287,United Kingdom
1970,536545,21134,NaN,1,2010-12-01 14:32:00,0.0,15287,United Kingdom
1971,536546,22145,NaN,1,2010-12-01 14:33:00,0.0,15287,United Kingdom
1972,536547,37509,NaN,1,2010-12-01 14:33:00,0.0,15287,United Kingdom
1987,536549,85226A,NaN,1,2010-12-01 14:34:00,0.0,15287,United Kingdom


In [8]:
df.loc[:, 'StockCode'].nunique(), df.loc[:, 'Description'].nunique()

(4070, 4223)

In [9]:
num_uniques = len(df.loc[:, ['StockCode', 'Description']].drop_duplicates(keep=False))
print(f'El número de productos únicos es {num_uniques}')

El número de productos únicos es 1429


In [10]:
import pandas as pd

# Supongamos que df es tu DataFrame
df1 = pd.DataFrame({
    'columna1': ['a', 'c', 'a', 'c', 'c'],
    'columna2': ['x', 'z', 'x', 'z', 'z']
})

# Eliminar filas duplicadas y obtener un nuevo DataFrame con filas únicas
df_filas_unicas = df1.drop_duplicates()

print("DataFrame original:")
print(df1)

print("\nDataFrame con filas únicas:")
print(df_filas_unicas)

print("\nDataFrame keep=False:")
print(df1.drop_duplicates(keep=False))


DataFrame original:
  columna1 columna2
0        a        x
1        c        z
2        a        x
3        c        z
4        c        z

DataFrame con filas únicas:
  columna1 columna2
0        a        x
1        c        z

DataFrame keep=False:
Empty DataFrame
Columns: [columna1, columna2]
Index: []
